In [62]:
from zat.log_to_dataframe import LogToDataFrame
import pandas as pd

In [31]:
log_to_df = LogToDataFrame()
conn_df = log_to_df.create_dataframe('zeeks/conn.log')
known_services_df = log_to_df.create_dataframe('zeeks/known_services.log')


In [28]:
print(conn_df.columns)
conn_df.head()

Index(['uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto',
       'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state',
       'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts',
       'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents',
       'ip_proto'],
      dtype='object')


,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,ip_proto
ts,,,,,,,,,,,,,,,,,,,,,
2017-12-09 18:04:31.275780916,C6OafO3QavazF9uqI2,10.131.66.31,137,10.131.66.255,137,udp,dns,0 days 00:00:08.466452,300,0,...,T,T,0,D,6,468,0,0,NaN,17
2017-12-09 18:09:08.530639887,CKIrTjbpVAOOeluW4,192.251.11.33,137,192.251.11.12,137,udp,dns,NaT,<NA>,<NA>,...,F,F,0,D^,1,96,0,0,NaN,17
2017-12-09 18:11:19.089591980,C8LjRO3HVlo6USqdo4,192.251.10.196,137,192.251.10.255,137,udp,dns,0 days 00:00:01.500664,150,0,...,F,F,0,D,3,234,0,0,NaN,17
2017-12-09 18:14:58.835217953,CLNc7G1rNVONwrsEKb,192.251.10.32,137,192.251.10.255,137,udp,dns,0 days 00:00:01.499757,150,0,...,F,F,0,D,3,234,0,0,NaN,17
2017-12-09 18:16:35.299012899,CXDRd43grE8K46XE02,192.251.10.32,137,192.251.10.255,137,udp,dns,0 days 00:00:01.499796,150,0,...,F,F,0,D,3,234,0,0,NaN,17


### Testing analysis

#### Check Ports
known_services.log filtered.

Previously: Since we don't have that currently, going to use conn.log and filter ports instead

Now: We do have it!


In [72]:
pd.DataFrame({'Port': conn_df['id.resp_p'].unique()}).sort_values(by='Port').to_html(index=False, justify="left").replace("\n", "")

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: left;">      <th>Port</th>    </tr>  </thead>  <tbody>    <tr>      <td>0</td>    </tr>    <tr>      <td>3</td>    </tr>    <tr>      <td>8</td>    </tr>    <tr>      <td>21</td>    </tr>    <tr>      <td>53</td>    </tr>    <tr>      <td>137</td>    </tr>    <tr>      <td>44818</td>    </tr>  </tbody></table>'

Using known_services:

In [73]:
unique_ks_df = known_services_df[['port_num', 'service']].groupby(['port_num', 'service'], observed=True).count().sort_values("port_num")
unique_ks_df

,
port_num,service
21,FTP
53,DNS
137,DNS
44818,(empty)


In [59]:
print(unique_ks_df.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
    </tr>
    <tr>
      <th>port_num</th>
      <th>service</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>21</th>
      <th>FTP</th>
    </tr>
    <tr>
      <th>53</th>
      <th>DNS</th>
    </tr>
    <tr>
      <th>137</th>
      <th>DNS</th>
    </tr>
    <tr>
      <th>44818</th>
      <th>(empty)</th>
    </tr>
  </tbody>
</table>


### check_external
Known outbound external connections https://github.com/esnet-security/zeek-outbound-known-services-with-origflag
